# 한국은행 의사록 어조 분석

이제 극성사전이 완성되었으므로, 이를 이용해 한국은행 의사록의 어조(tone)을 측정하도록 한다. 

우선 각 일자의 한국은행 의사록을 `sentences`로 분리한 뒤, 각 `sentences`를 다시 `ngrams`로 분리한다. 

문장별로 추출한 `ngrams`를 바탕으로 각 문장의 `tone_sentence`를 계산한다. 

문장별 `tone_sentence`가 계산됐으면, 문장을 다시 합쳐 문서(의사록)의 `tone_doc`을 구한다. 

한 날짜에는 한 문서가 있고, 따라서 한 `tone_doc`이 있으므로, 이제 이 데이터를 바탕으로 한국은행 기준금리 데이터와의 상관관계와 그래프를 확인한다. 

In [30]:
# -*- coding: utf-8 -*-

import pandas as pd
import multiprocessing as mp

import preprocessing_mp_worker as worker

## 의사록 데이터를 불러온다. 

In [73]:
minutes_df = pd.read_pickle('./data_files/BOK_minutes/pkl/minutes_ngram.pkl')
minutes_df = minutes_df[['date', 'minutes']]
minutes_df.head()

,date,minutes
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2005-06-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,2005-07-07,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,2005-08-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,2005-09-08,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


## 각 의사록(`minutes`)을 문장(`sentences`)들로 분리한다. 

In [75]:
%%time

minutes_df.loc[:, 'sentences'] = minutes_df.minutes.map(lambda x: worker.tidy_sentences(x)[0])

Wall time: 770 ms


In [78]:
minutes_df = minutes_df.explode('sentences')
minutes_df.head()

,date,minutes,sentences
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,130.1...
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,5. 참 여 자 이 상 용 감사 정 규 영 ...
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"(３) 위원 토의내용 ｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 소비..."
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,동 위원은 서울 강남에서 시작된 부동산가격 상승이 다른 지역으로 확산될 기미가 ...
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,또한 동 위원은 정부가 여러 차례에 걸쳐 부동산시장 안정대책을 내놓았음.


## 각 문장을 `ngrams`로 분리한다. 

In [80]:
def mp_text2ngram(sentence):
    with mp.Pool(processes=6) as p:
        mp_result = p.map(worker.text2ngram, sentence)
    return mp_result

In [ ]:
%%time

minutes_df.loc[:, 'ngrams'] = minutes_df.sentences.map(mp_text2ngram)